In [1]:
from token_creator import get_creds
get_creds()

' token available '

In [1]:
from gmail_agent import gmail_agent
from calendar_agent import Calendar_agent
from maps_agent import maps_agent
from contacts_agent import contacts_agent
from tasks_agent import tasks_agent
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.tools import tool,Tool
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition,InjectedState
from langchain_core.messages import (
    HumanMessage,
    ToolMessage,
    
)
from langgraph.types import Command
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.tools.base import InjectedToolCallId


from typing_extensions import TypedDict
from typing import Annotated

import os
from dotenv import load_dotenv 

#get graph visuals
from IPython.display import Image, display
from langchain_core.runnables.graph import MermaidDrawMethod
import os
import requests
import gradio as gr


In [2]:
load_dotenv()
GOOGLE_API_KEY=os.getenv('google_api_key')
pse=os.getenv('pse')

In [3]:
GEMINI_MODEL='gemini-2.0-flash'
llm = ChatGoogleGenerativeAI(google_api_key=GOOGLE_API_KEY, model=GEMINI_MODEL, temperature=0.3)

In [4]:
mail_agent=gmail_agent(llm)
calendar_agent=Calendar_agent(llm)
maps_agent=maps_agent(llm)
tasks_agent=tasks_agent(llm)
contacts_agent=contacts_agent(llm)

In [5]:
class State(TypedDict):
    messages: Annotated[list, add_messages]

In [6]:
def google_image_search(query: str) -> str:
  """Search for images using Google Custom Search API
  args: query
  return: image url
  """
  # Define the API endpoint for Google Custom Search
  url = "https://www.googleapis.com/customsearch/v1"

  params = {
      "q": query,
      "cx": pse,
      "key": GOOGLE_API_KEY,
      "searchType": "image",  # Search for images
      "num": 1  # Number of results to fetch
  }

  # Make the request to the Google Custom Search API
  response = requests.get(url, params=params)
  data = response.json()

  # Check if the response contains image results
  if 'items' in data:
      # Extract the first image result
      image_url = data['items'][0]['link']
      return image_url
  else:
      return "Sorry, no images were found for your query."

google_image_tool=Tool(name='google_image_tool', func=google_image_search, description='Use this tool to search for images using Google Custom Search API')

In [7]:
# Define the API endpoint for Google Custom Search
url = "https://www.googleapis.com/customsearch/v1"

params = {
    "q": 'cat',
    "cx": pse,
    "key": GOOGLE_API_KEY,
    "searchType": "image",  # Search for images
    "num": 1  # Number of results to fetch
}

# Make the request to the Google Custom Search API
response = requests.get(url, params=params)
data = response.json()

# Check if the response contains image results
if 'items' in data:
    # Extract the first image result
    image_url = data['items'][0]['link']


In [8]:
data

{'kind': 'customsearch#search',
 'url': {'type': 'application/json',
  'template': 'https://www.googleapis.com/customsearch/v1?q={searchTerms}&num={count?}&start={startIndex?}&lr={language?}&safe={safe?}&cx={cx?}&sort={sort?}&filter={filter?}&gl={gl?}&cr={cr?}&googlehost={googleHost?}&c2coff={disableCnTwTranslation?}&hq={hq?}&hl={hl?}&siteSearch={siteSearch?}&siteSearchFilter={siteSearchFilter?}&exactTerms={exactTerms?}&excludeTerms={excludeTerms?}&linkSite={linkSite?}&orTerms={orTerms?}&dateRestrict={dateRestrict?}&lowRange={lowRange?}&highRange={highRange?}&searchType={searchType}&fileType={fileType?}&rights={rights?}&imgSize={imgSize?}&imgType={imgType?}&imgColorType={imgColorType?}&imgDominantColor={imgDominantColor?}&alt=json'},
 'queries': {'request': [{'title': 'Google Custom Search - cat',
    'totalResults': '61320000000',
    'searchTerms': 'cat',
    'count': 1,
    'startIndex': 1,
    'inputEncoding': 'utf8',
    'outputEncoding': 'utf8',
    'safe': 'off',
    'cx': '3306

In [9]:
@tool 
def contacts_manager(query: str):
    """use this tool to answer queries about a contact or a person
    this tool can:
    list my contacts
    get a contact's details
    delete a contact
    create a contact
    modify a contact
    args: query - pass the email related queries directly here
    """
    response=contacts_agent.chat(str(query))
    return response

In [10]:
@tool
def tasks_manager(query:str):
    """use this tool to answer task related queries
    this tool can:
    list tasks
    create tasks
    get task details
    complete a task (which also deletes it :) )

    args: query - pass the email related queries directly here
    
    """


    response=tasks_agent.chat(str(query))
    return response

In [11]:
@tool
def maps_manager(query:str):
    """tool to use to answer maps and location queries
    this tool can:
    find locations such as restorants, bowling alleys, museums and others
    display those locations's infos (eg. adress, name, url, price range)
    args: query - pass the email related queries directly here
    """
    response=maps_agent.chat(str(query))
    return response

In [12]:
@tool
def mail_manager(query:str):
    """Tool to use to answer any email related queries
    this tool can:
    get new mail
    show the inbox
    get a specific mail's content to display
    create a draft of a new mail
    show the current draft
    send the draft
    list all the drafts
    args: query - pass the email related queries directly here
    """
    response=mail_agent.chat(str(query))
    return response

In [13]:
@tool
def calendar_manager(query:str):
    """tool to use to answere any calendar or schedule related queries
    this tool can:
    create recuring events
    create quick events
    refresh the calendar
    show the calendar
    args: query - pass the calendar related queries directly here
    """
    response=calendar_agent.chat(str(query))
    return response

In [14]:
class google_agent:
    def __init__(self,llm: any):
        self.agent=self._setup(llm)
    def _setup(self,llm):
   
        langgraph_tools=[mail_manager,calendar_manager,maps_manager, tasks_manager, contacts_manager, google_image_tool]



        graph_builder = StateGraph(State)

        # Modification: tell the LLM which tools it can call
        llm_with_tools = llm.bind_tools(langgraph_tools)
        tool_node = ToolNode(tools=langgraph_tools)
        def chatbot(state: State):
            """ travel assistant that answers user questions about their trip.
            Depending on the request, leverage which tools to use if necessary."""
            return {"messages": [llm_with_tools.invoke(state['messages'])]}

        graph_builder.add_node("chatbot", chatbot)

        
        graph_builder.add_node("tools", tool_node)
        # Any time a tool is called, we return to the chatbot to decide the next step
        graph_builder.add_edge(START,'chatbot')
        graph_builder.add_edge("tools", "chatbot")
        graph_builder.add_conditional_edges(
            "chatbot",
            tools_condition,
        )
        memory=MemorySaver()
        graph=graph_builder.compile(checkpointer=memory)
        return graph
        

    def display_graph(self):
        return display(
                        Image(
                                self.agent.get_graph().draw_mermaid_png(
                                    draw_method=MermaidDrawMethod.API,
                                )
                            )
                        )
    def stream(self,input:str):
        config = {"configurable": {"thread_id": "1"}}
        input_message = HumanMessage(content=input)
        for event in self.agent.stream({"messages": [input_message]}, config, stream_mode="values"):
            event["messages"][-1].pretty_print()

    def chat(self,input:str):
        config = {"configurable": {"thread_id": "1"}}
        response=self.agent.invoke({'messages':HumanMessage(content=str(input))},config)
        return response['messages'][-1].content
    
    def get_state(self, state_val:str):
        config = {"configurable": {"thread_id": "1"}}
        return self.agent.get_state(config).values[state_val]

In [15]:
agent=google_agent(llm)

In [16]:
def chatbot(input,history):
    response=agent.chat(input)
    return response


In [17]:
demo = gr.ChatInterface(chatbot, type="messages", autofocus=False)

if __name__ == "__main__":
    demo.launch()

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [17]:
agent.get_state('messages')

KeyError: 'messages'